# Import Relevant Libraries

In [1]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup

# Scrape the data

In [2]:
def scrape_articles(page_number, flag):
    url_params = {'Text' : 'accidents', 'StartRowIndex' : page_number}
    url = 'https://english.ahram.org.eg/UI/Front/Search.aspx'
    response = requests.get(url=url, params=url_params) 
    soup = BeautifulSoup(response.text, 'html.parser')
    end = 19 if page_number == 180 else 20
    articles = []
    for i in range(end):
        article = soup.find(id=f'ContentPlaceHolder1_dlNewsContent_{"hyplnkTitle" if flag else "lblCreationDate"}_{i}').get_text()
        articles.append(article if flag else article.split(' ')[0])  
    return articles

In [3]:
articles = []
dates = []
for i in range(0, 181, 20):
    articles += scrape_articles(i, True)
    dates += scrape_articles(i, False)

## Removing Unnecessary Characters

In [4]:
articles = [re.sub('\s+', ' ', article) for article in articles]
articles

['Campaigning for road safety',
 'One killed, 8 injured in Cairo Ring Road collision ',
 'At least 30 dead in migrant boat accidents in Greece',
 'Egypt witnesses 28.9% drop in traffic injuries and 8.9% decrease in deaths in 2020',
 "After 'Rust' shooting, a look at other notable set accidents",
 'Egypt, Austria ink €8.5 mln agreement to support railway sector',
 'Heavy rain in India triggers floods, landslides; at least 125 dead',
 'At least 27 killed in two road accidents in Algeria',
 'Transport minister vows to ensure safe travel for railway passengers',
 "Transport minister seeks legislation to fire 'extremists' working in railway sector",
 "Minister of Transportation reshuffles Egypt's Railways Authority management in wake of recent train accidents",
 '15 injured in train derailment in Sharqiya in the Nile Delta',
 'Twenty people killed, three injured in a road accident in Assiut ',
 'Probing the Sohag train crash',
 "Modernising Egypt's railway system only way to end accidents: 

In [5]:
df = pd.DataFrame(data={'accident': articles, 'date': dates})
df

,accident,date
0,Campaigning for road safety,1/28/2022
1,"One killed, 8 injured in Cairo Ring Road colli...",1/5/2022
2,At least 30 dead in migrant boat accidents in ...,12/25/2021
3,Egypt witnesses 28.9% drop in traffic injuries...,11/10/2021
4,"After 'Rust' shooting, a look at other notable...",10/23/2021
...,...,...
194,Three accidents north of Cairo result in 74 in...,1/13/2011
195,Egypt: Death toll in Sharkiya accidents rises ...,1/13/2011
196,Two dozen dead in Sharkiya accidents,1/11/2011
197,Three dead and 15 injured in road accidents in...,1/9/2011


## Converting the date column to pandas datetime object

In [9]:
df['date'] = pd.to_datetime(df['date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   accident  199 non-null    object        
 1   date      199 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 3.2+ KB


In [10]:
df.to_csv('accidents_alahram.csv')